In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/CarloSgaravatti/Recsys_Challenge_2023.git

Cloning into 'Recsys_Challenge_2023'...
remote: Enumerating objects: 303, done.
remote: Counting objects: 100% (303/303), done.
remote: Compressing objects: 100% (264/264), done.
remote: Total 303 (delta 124), reused 170 (delta 32), pack-reused 0
Receiving objects: 100% (303/303), 11.75 MiB | 14.79 MiB/s, done.
Resolving deltas: 100% (124/124), done.


In [2]:
! pip install PyGithub requests

  Obtaining dependency information for PyGithub from https://files.pythonhosted.org/packages/be/04/810d131be173cba445d3658a45512b2b2b3d0960d52c4a300d6ec5e00f52/PyGithub-2.1.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 14.3 MB/s eta 0:00:00


In [3]:
! cd /kaggle/working/Recsys_Challenge_2023 && python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorizationImpressions_Cython_Epoch.c:1109:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorizationImpressions_Cython_Epoch.c: In functio

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import random
import tqdm
import gc
import time
import os
import shutil
import optuna
import scipy.sparse as sps

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/hybrid/tuning_hybrid.db
/kaggle/input/hybrid/__results__.html
/kaggle/input/hybrid/__notebook__.ipynb
/kaggle/input/hybrid/best_params_hybrid.json
/kaggle/input/hybrid/__output__.json
/kaggle/input/hybrid/custom.css
/kaggle/input/hybrid/Recsys_Challenge_2023/run_all_algorithms.py
/kaggle/input/hybrid/Recsys_Challenge_2023/cython_example_FunkSVD.py
/kaggle/input/hybrid/Recsys_Challenge_2023/LICENSE
/kaggle/input/hybrid/Recsys_Challenge_2023/.gitignore
/kaggle/input/hybrid/Recsys_Challenge_2023/README.md
/kaggle/input/hybrid/Recsys_Challenge_2023/run_hyperparameter_search.py
/kaggle/input/hybrid/Recsys_Challenge_2023/cython_example_SLIM_MSE.py
/kaggle/input/hybrid/Recsys_Challenge_2023/requirements.txt
/kaggle/input/hybrid/Recsys_Challenge_2023/run_test_recommenders.py
/kaggle/input/hybrid/Recsys_Challenge_2023/run_compile_all_cython.py
/kaggle/input/hybrid/Recsys_Challenge_2023/hybrid.ipynb
/kaggle/input/hybrid/Recsys_Challenge_2023/Data_manager/DataSplitter_Holdout.py
/ka

In [5]:
%cd /kaggle/working/Recsys_Challenge_2023

/kaggle/working/Recsys_Challenge_2023


In [6]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

# Configurations

In [7]:
from github import Github
from github import Auth

auth = Auth.Token(token)
g = Github(auth=auth)

repo = None
for r in g.get_user().get_repos():
    if r.name == 'Recsys_Challenge_2023':
        repo = r
        print('Repository found')

Repository found


In [8]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    try:
        contents = repo.get_contents(filepath_github)
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(contents.path, commit_message, file.read(), contents.sha)
    except Exception:
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(filepath_github, commit_message, file.read())

In [9]:
config = {
    'tune_parameters': True,
    'database_path': '/kaggle/working/tuning_hybrid.db',
    'copy_prev_tuning_db': True,
    'copy_prev_best_params': True,
    'save_github': True
}

# Tuning

In [10]:
from Data_manager.IncrementalSparseMatrix import IncrementalSparseMatrix


class CrossValidationSplitter:
    def __init__(self, n_splits):
        self.n_splits = n_splits
        self.folds = []
        
    def split(self, URM):
        n_int_per_split = int(URM.nnz / self.n_splits)
        indices = np.arange(0, URM.nnz, 1)
        np.random.shuffle(indices)
        num_users, num_items = URM.shape
        
        URM_all = sps.coo_matrix(URM)
        
        for i in range(self.n_splits):
            if i == self.n_splits - 1:
                indices_valid = indices[n_int_per_split * i:]
                indices_train = indices[:n_int_per_split * i]
            else:
                indices_valid = indices[n_int_per_split * i: n_int_per_split * (i + 1)]
                indices_train_1 = indices[n_int_per_split * max(0, i - 1): n_int_per_split * i]
                indices_train_2 = indices[n_int_per_split * (i + 1):]
                indices_train = np.concatenate((indices_train_1, indices_train_2))
                
            URM_valid_builder = IncrementalSparseMatrix(
                n_rows=num_users, n_cols=num_items, auto_create_col_mapper=False, 
                auto_create_row_mapper=False
            )
            
            URM_train_builder = IncrementalSparseMatrix(
                n_rows=num_users, n_cols=num_items, auto_create_col_mapper=False, 
                auto_create_row_mapper=False
            )
            
            URM_valid_builder.add_data_lists(
                URM_all.row[indices_valid],
                URM_all.col[indices_valid],
                URM_all.data[indices_valid]
            )
            
            URM_train_builder.add_data_lists(
                URM_all.row[indices_train],
                URM_all.col[indices_train],
                URM_all.data[indices_train]
            )
            
            yield URM_train_builder.get_SparseMatrix(), URM_valid_builder.get_SparseMatrix()

In [11]:
train_data = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv')
user_ids_test = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv')

In [12]:
URM_all = sps.csr_matrix((train_data.data.values,
                          ((train_data.row - 1).values, (train_data.col - 1).values)),
                        shape=(train_data.row.max(), train_data.col.max()))
URM_all

<13024x22347 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [13]:
URM_train, URM_test_complete = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_validation, URM_test = split_train_in_two_percentage_global_sample(URM_test_complete, train_percentage = 0.8)

## Pipeline Hybrid Recommender

In [14]:
class HybridRecommender:
    def __init__(self, fine_recommender, coarse_recommender, coarse_cutoff):
        self.fine_recommender = fine_recommender
        self.coarse_recommender = coarse_recommender
        self.coarse_cutoff = coarse_cutoff
        
    def recommend(self, user_ids, cutoff=10, remove_seen_flag=True, remove_top_pop_flag=False, return_scores=True, remove_custom_items_flag=True):
        coarse_recommendations = self.coarse_recommender.recommend(
            user_ids,
            cutoff=self.coarse_cutoff, 
            remove_seen_flag=remove_seen_flag, 
            remove_top_pop_flag=remove_top_pop_flag, 
            return_scores=False, 
            remove_custom_items_flag=remove_custom_items_flag
        )
        
        final_recommendations = []
        final_scores = []
        for i in range(len(user_ids)):
            recommendations, scores = self.fine_recommender.recommend(
                [user_ids[i]],
                cutoff=cutoff,
                items_to_compute=coarse_recommendations[i], 
                remove_seen_flag=remove_seen_flag, 
                remove_top_pop_flag=remove_top_pop_flag, 
                return_scores=True, 
                remove_custom_items_flag=remove_custom_items_flag
            )
            final_recommendations.append(recommendations[0])
            final_scores.append(scores[0])
        if return_scores:
            return np.array(final_recommendations), np.array(final_scores)
        return np.array(final_recommendations)
    
    def get_URM_train(self):
        return URM_train

In [15]:
from concurrent.futures import ThreadPoolExecutor


def evaluate_hybrid_recommender(
    topK_fine, shrink_fine, similarity_fine, topK_coarse, shrink_coarse, 
    similarity_coarse, coarse_cutoff , URM_train, URM_valid
):
    recommender_item_knn = UserKNNCFRecommender(URM_train)
    recommender_item_knn.fit(topK=topK_coarse, shrink=shrink_coarse, similarity=similarity_coarse)

    recommender_user_knn = ItemKNNCFRecommender(URM_train)
    recommender_user_knn.fit(topK=topK_fine, shrink=shrink_fine, similarity=similarity_fine)

    hybrid_user_item_knn = HybridRecommender(recommender_item_knn, recommender_user_knn, coarse_cutoff)

    evaluator = EvaluatorHoldout(URM_valid, cutoff_list=[10])
    result_df, _ = evaluator.evaluateRecommender(hybrid_user_item_knn)
    print(f'MAP is {result_df.loc[10, "MAP"]}')
    return result_df.loc[10, 'MAP']



def objective(trial):
    topK_fine = trial.suggest_int('topK_fine', 10, 1000, log=True)
    shrink_fine = trial.suggest_int('shrink_fine', 0, 500)
    similarity_fine = trial.suggest_categorical('similarity_fine', ["cosine", "asymmetric", "dice", "jaccard", "tanimoto", "tversky"])
    topK_coarse = trial.suggest_int('topK_coarse', 10, 1000, log=True)
    shrink_coarse = trial.suggest_int('shrink_coarse', 0, 500)
    similarity_coarse = trial.suggest_categorical('similarity_coarse', ["cosine", "asymmetric", "dice", "jaccard", "tanimoto", "tversky"])
    coarse_cutoff = trial.suggest_int('coarse_cutoff', 10, 1000, log=True)
    
    map_sum = 0
    
    results = [None] * 10
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []
        for URM_train, URM_valid in CrossValidationSplitter(10).split(URM_all):
            futures.append(
                executor.submit(evaluate_hybrid_recommender, topK_fine, shrink_fine, 
                                similarity_fine, topK_coarse, shrink_coarse, similarity_coarse, 
                                coarse_cutoff, URM_train, URM_valid)
            )
        for i, future in enumerate(futures):
            results[i] = future.result()
    return np.mean(results)

In [16]:
try:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/tuning_hybrid.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

In [17]:
if config['tune_parameters']:
    study = optuna.create_study(direction='maximize', study_name='hyperparameters_tuning_hybrid_knn', 
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)
    study.optimize(objective, n_trials=50)

[I 2023-11-21 16:48:22,532] Using an existing study with name 'hyperparameters_tuning_hybrid_knn' instead of creating a new one.


UserKNNCFRecommender: URM Detected 500 ( 3.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 172 ( 0.8%) items with no interactions.
UserKNNCFRecommender: URM Detected 494 ( 3.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 180 ( 0.8%) items with no interactions.
UserKNNCFRecommender: URM Detected 597 ( 4.6%) users with no interactions.
UserKNNCFRecommender: URM Detected 235 ( 1.1%) items with no interactions.
UserKNNCFRecommender: URM Detected 767 ( 5.9%) users with no interactions.
UserKNNCFRecommender: URM Detected 351 ( 1.6%) items with no interactions.
UserKNNCFRecommender: URM Detected 925 ( 7.1%) users with no interactions.
UserKNNCFRecommender: URM Detected 568 ( 2.5%) items with no interactions.
UserKNNCFRecommender: URM Detected 1151 ( 8.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 864 ( 3.9%) items with no interactions.
UserKNNCFRecommender: URM Detected 1441 (11.1%) users with no interactions.
UserKNNCFRecommender: U

[I 2023-11-21 16:53:50,079] Trial 50 finished with value: 0.015329041802160112 and parameters: {'topK_fine': 34, 'shrink_fine': 216, 'similarity_fine': 'dice', 'topK_coarse': 215, 'shrink_coarse': 253, 'similarity_coarse': 'tversky', 'coarse_cutoff': 12}. Best is trial 48 with value: 0.015487193289739493.


MAP is 0.018703656745591508
UserKNNCFRecommender: URM Detected 485 ( 3.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 172 ( 0.8%) items with no interactions.
UserKNNCFRecommender: URM Detected 487 ( 3.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 172 ( 0.8%) items with no interactions.
UserKNNCFRecommender: URM Detected 612 ( 4.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 239 ( 1.1%) items with no interactions.
UserKNNCFRecommender: URM Detected 749 ( 5.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 348 ( 1.6%) items with no interactions.
UserKNNCFRecommender: URM Detected 943 ( 7.2%) users with no interactions.
UserKNNCFRecommender: URM Detected 542 ( 2.4%) items with no interactions.
UserKNNCFRecommender: URM Detected 1138 ( 8.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 832 ( 3.7%) items with no interactions.
UserKNNCFRecommender: URM Detected 1455 (11.2%) users with no interacti

[I 2023-11-21 16:58:33,707] Trial 51 finished with value: 0.015239337401799577 and parameters: {'topK_fine': 42, 'shrink_fine': 212, 'similarity_fine': 'dice', 'topK_coarse': 203, 'shrink_coarse': 259, 'similarity_coarse': 'tversky', 'coarse_cutoff': 12}. Best is trial 48 with value: 0.015487193289739493.


...

[I 2023-11-21 20:56:42,348] Trial 97 finished with value: 0.016933853706988414 and parameters: {'topK_fine': 13, 'shrink_fine': 25, 'similarity_fine': 'tversky', 'topK_coarse': 516, 'shrink_coarse': 413, 'similarity_coarse': 'tanimoto', 'coarse_cutoff': 11}. Best is trial 94 with value: 0.016967118324877104.


UserKNNCFRecommender: URM Detected 480 ( 3.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 165 ( 0.7%) items with no interactions.
UserKNNCFRecommender: URM Detected 482 ( 3.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 186 ( 0.8%) items with no interactions.
UserKNNCFRecommender: URM Detected 624 ( 4.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 233 ( 1.0%) items with no interactions.
UserKNNCFRecommender: URM Detected 737 ( 5.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 360 ( 1.6%) items with no interactions.
UserKNNCFRecommender: URM Detected 910 ( 7.0%) users with no interactions.
UserKNNCFRecommender: URM Detected 554 ( 2.5%) items with no interactions.
UserKNNCFRecommender: URM Detected 1139 ( 8.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 892 ( 4.0%) items with no interactions.
UserKNNCFRecommender: URM Detected 1421 (10.9%) users with no interactions.
UserKNNCFRecommender: U

[I 2023-11-21 21:01:39,835] Trial 98 finished with value: 0.0167015663513181 and parameters: {'topK_fine': 10, 'shrink_fine': 54, 'similarity_fine': 'tversky', 'topK_coarse': 512, 'shrink_coarse': 416, 'similarity_coarse': 'tanimoto', 'coarse_cutoff': 11}. Best is trial 94 with value: 0.016967118324877104.


MAP is 0.023337214697668165
UserKNNCFRecommender: URM Detected 496 ( 3.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 169 ( 0.8%) items with no interactions.
UserKNNCFRecommender: URM Detected 480 ( 3.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 174 ( 0.8%) items with no interactions.
UserKNNCFRecommender: URM Detected 621 ( 4.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
UserKNNCFRecommender: URM Detected 757 ( 5.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 378 ( 1.7%) items with no interactions.
UserKNNCFRecommender: URM Detected 910 ( 7.0%) users with no interactions.
UserKNNCFRecommender: URM Detected 531 ( 2.4%) items with no interactions.
UserKNNCFRecommender: URM Detected 1163 ( 8.9%) users with no interactions.
UserKNNCFRecommender: URM Detected 900 ( 4.0%) items with no interactions.
UserKNNCFRecommender: URM Detected 1455 (11.2%) users with no interacti

[I 2023-11-21 21:06:31,622] Trial 99 finished with value: 0.01583461111626587 and parameters: {'topK_fine': 10, 'shrink_fine': 0, 'similarity_fine': 'tversky', 'topK_coarse': 532, 'shrink_coarse': 415, 'similarity_coarse': 'tanimoto', 'coarse_cutoff': 11}. Best is trial 94 with value: 0.016967118324877104.


In [18]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

In [19]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_param_importances(study)
    fig.show()

In [20]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_slice(study)
    fig.show()

In [21]:
if config['tune_parameters']:
    with open('/kaggle/working/best_params_hybrid.json', 'w') as params_file:
        json.dump(study.best_params, params_file)
        
    if config['save_github']: 
        upload_file(
            '/kaggle/working/best_params_hybrid.json', 
            'tuning_results/best_params_hybrid.json', 
            'Hybrid User-Item KNN tuning results (from kaggle notebook)'
        )
    
elif config['copy_prev_best_params']:
    shutil.copyfile(
        '/kaggle/input/hyperparameters-tuning/best_params_hybrid.json', 
        '/kaggle/working/best_params_hybrid.json'
    )

In [22]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        'tuning_results/tuning_hybrid.db', 
        'Hybrid recommenders tuning db updated results (from kaggle notebook)'
    )

## Merging Models

In [23]:
# with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_knn_item.json', 'r') as params_file:
#     params_item_knn1 = json.load(params_file)
# params_item_knn1

In [24]:
# params_item_knn2 = {'similarity': 'jaccard', 'topK': 47, 'shrink': 31}

In [25]:
# recommender_item_knn_1 = ItemKNNCFRecommender(URM_train)
# recommender_item_knn_1.fit(topK=params_item_knn1['topK'], shrink=params_item_knn1['shrink'], similarity=params_item_knn1['similarity'])

# recommender_item_knn_2 = ItemKNNCFRecommender(URM_train)
# recommender_item_knn_2.fit(topK=params_item_knn2['topK'], shrink=params_item_knn2['shrink'], similarity=params_item_knn2['similarity'])

In [26]:
# from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender

# hybrid_similarity = ItemKNNSimilarityHybridRecommender(URM_train, recommender_item_knn_1.W_sparse, recommender_item_knn_2.W_sparse)
# hybrid_similarity.fit(topK=50, alpha=0.5)

In [27]:
# evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])

# result_df, _ = evaluator.evaluateRecommender(recommender_item_knn_1)
# display(result_df)
# result_df, _ = evaluator.evaluateRecommender(recommender_item_knn_2)
# display(result_df)
# result_df, _ = evaluator.evaluateRecommender(hybrid_similarity)
# display(result_df)

# Training best model

In [28]:
# with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_knn_item.json', 'r') as params_file:
#     params_item_knn = json.load(params_file)
    
# with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_knn_user.json', 'r') as params_file:
#     params_user_knn = json.load(params_file)
    
# params_item_knn, params_user_knn

In [29]:
# recommender_item_knn = UserKNNCFRecommender(URM_train)
# recommender_item_knn.fit(topK=params_user_knn['topK'], shrink=params_user_knn['shrink'], similarity=params_user_knn['similarity'])

# recommender_user_knn = ItemKNNCFRecommender(URM_train)
# recommender_user_knn.fit(topK=params_item_knn['topK'], shrink=params_item_knn['shrink'], similarity=params_item_knn['similarity'])

In [30]:
# recommendations = hybrid_user_item_knn.recommend(user_ids_test['user_id'].unique() - 1, cutoff=10, return_scores=False)
# recommendations = [' '.join([str(item + 1) for item in r]).strip() for r in recommendations]

# submission_df = pd.DataFrame({'user_id': user_ids_test['user_id'].unique(), 'item_list': recommendations})
# submission_df.head()

In [31]:
# evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])

# hybrid_user_item_knn = HybridRecommender(recommender_item_knn, recommender_user_knn, 11)
# result_df, _ = evaluator.evaluateRecommender(hybrid_user_item_knn)
# result_df

In [32]:
# submission_df.to_csv('/kaggle/working/submission.csv', index=False)